In [1]:
# Calling the necessary libraries
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width',1000)

### 1. Business Problem
FLO wants its customers to be segmented and marketing strategies to be determined according to these segments. For this purpose, customers' behaviors will be defined and groups will be formed according to these behavioral clusters.

### 2. Dataset Story
The data set consists of information obtained from the past shopping behavior of customers who made their last purchases as OmniChannel (both online and offline shoppers) in 2020 - 2021.

### 3. Features
+ master_id - Unique customer identifier
+ order_channel - Indication of the platform used for shopping (Android, iOS, Desktop, Mobile)
+ last_order_channel - The channel used for the most recent purchase
+ first_order_date - Date of the customer's initial purchase
+ last_order_date - Date of the customer's most recent purchase
+ last_order_date_online - Date of the customer's last online purchase
+ last_order_date_offline - Date of the customer's last offline purchase
+ order_num_total_ever_online - Total number of purchases made online by the customer
+ order_num_total_ever_offline - Total number of purchases made offline by the customer
+ customer_value_total_ever_offline - Total amount spent by the customer in offline purchases
+ customer_value_total_ever_online - Total amount spent by the customer in online purchases
+ interested_in_categories_12 - List of categories in which the customer has made purchases in the last 12 months

### 4.Databehandling

In [2]:
# Load the dataset
df_ = pd.read_csv("flo_data_20k.csv")

In [3]:
df = df_.copy()
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]


In [4]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date', 'last_order_date', 'last_order_date_online', 'last_order_date_offline', 'order_num_total_ever_online', 'order_num_total_ever_offline', 'customer_value_total_ever_offline', 'customer_value_total_ever_online', 'interested_in_categories_12'], dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [6]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [7]:
df.shape

(19945, 12)

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.11,4.23,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.00,1.91,2.06,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.00,253.92,301.53,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.00,497.32,832.60,12.99,149.98,286.46,578.44,45220.13


#### 4.1 Data Cleaning and Preprocessing

In [9]:
# Calculating the total customer value and number of orders for each customer
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [10]:
# Convert date columns to datetime format
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [11]:
# Distribution of the number of customers in shopping channels, total number of products purchased and total expenditures
df.groupby("order_channel").agg({"master_id":"count",
                                 "order_num_total":"sum",
                                 "customer_value_total":"sum"})

,master_id,order_num_total,customer_value_total
order_channel,,,
Android App,9495,52269.00,7819062.76
Desktop,2735,10920.00,1610321.46
Ios App,2833,15351.00,2525999.93
Mobile,4882,21679.00,3028183.16


In [12]:
# Top 10 highest earning customers
df.sort_values("customer_value_total", ascending=False)[:10]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00,2.00,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00,45905.10
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.00,1.00,130.49,36687.80,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.00,36818.29
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00,1.00,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00,33918.10
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.00,1.00,538.94,30688.47,"[ERKEK, KADIN, AKTIFSPOR]",11.00,31227.41
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.00,3.00,18119.14,2587.20,[AKTIFSPOR],4.00,20706.34
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00,4.00,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.00,18443.57
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00,1.00,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00,16918.57
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.00,1.00,180.73,12545.37,"[ERKEK, AKTIFSPOR]",37.00,12726.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00,1.00,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00,12282.24
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.00,2.00,64.97,12038.18,"[ERKEK, KADIN]",20.00,12103.15


In [13]:
#  Top 10 customers with the most orders
df.sort_values("order_num_total", ascending=False)[:10]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00,2.00,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00,45905.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00,1.00,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00,12282.24
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.00,109.00,10239.46,143.98,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.00,10383.44
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.00,87.00,8432.25,139.98,[ERKEK],88.00,8572.23
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.00,81.00,3997.55,242.81,[ERKEK],83.00,4240.36
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00,1.00,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00,33918.10
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.00,74.00,4640.77,543.28,"[KADIN, AKTIFSPOR]",77.00,5184.05
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.00,3.00,292.93,5004.95,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.00,5297.88
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00,1.00,82.48,16836.09,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00,16918.57
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00,4.00,843.68,17599.89,"[ERKEK, KADIN, AKTIFSPOR]",70.00,18443.57


#### 4.2 Calculation of RFM metrics

In [14]:
# Determining the date of analysis
df["last_order_date"].max() # 2021-05-30
analysis_date = dt.datetime(2021,6,1)

In [15]:
df["customer_id"] = df["master_id"]

In [16]:
rfm = df.groupby("customer_id").agg({"last_order_date": lambda last_order_date:(analysis_date - last_order_date).dt.days,
                                    "order_num_total": "sum",
                                    "customer_value_total": "sum"})

In [17]:
rfm.reset_index(inplace=True)

In [18]:
rfm.head()

,customer_id,last_order_date,order_num_total,customer_value_total
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00,776.07
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00,269.47
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.00,722.69
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00,874.16
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00,1620.33


In [19]:
rfm.columns = ["customer_id","recency", "frequency", "monetary"]

In [20]:
rfm.head()

,customer_id,recency,frequency,monetary
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00,776.07
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00,269.47
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.00,722.69
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00,874.16
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00,1620.33


In [21]:
# Calculate RFM scores
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

# expressing recency_score and frequency_score as a single variable and saving as RF_SCORE
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))

In [22]:
rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00,776.07,5,4,4,54
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00,269.47,1,2,1,12
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.00,722.69,2,3,4,23
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00,874.16,5,4,4,54
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00,1620.33,5,4,5,54


In [23]:
# Defining segments to make the generated RFM scores more explainable and translating RF_SCORE into segments 
# with the help of the defined seg_map
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [24]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [25]:
rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00,776.07,5,4,4,54,champions
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00,269.47,1,2,1,12,hibernating
2,000be838-85df-11ea-a90b-000d3a38a36f,213,4.00,722.69,2,3,4,23,at_Risk
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00,874.16,5,4,4,54,champions
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00,1620.33,5,4,5,54,champions


In [26]:
# Examination of recency, frequnecy and monetary averages of segments
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])


recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       114.03  1643      2.41  1643   361.65  1643
at_Risk              242.33  3152      4.47  3152   648.33  3152
cant_loose           235.16  1194     10.72  1194  1481.65  1194
champions             17.14  1920      8.97  1920  1410.71  1920
hibernating          247.43  3589      2.39  3589   362.58  3589
loyal_customers       82.56  3375      8.36  3375  1216.26  3375
need_attention       113.04   806      3.74   806   553.44   806
new_customers         17.98   673      2.00   673   344.05   673
potential_loyalists   36.87  2925      3.31  2925   533.74  2925
promising             58.69   668      2.00   668   334.15   668

FLO is adding a new women's footwear brand to its collection. The product prices of this new brand are above the general customer preferences. For this reason, it is desired to specially contact customers who will be interested in promoting the brand and product sales. The target audience is planned to be loyal customers who shop frequently, especially in the women's category.

In [27]:
loyal_customers  = rfm[rfm["segment"].isin(["champions","loyal_customers"])]["customer_id"]
female_customers  = df[(df["master_id"].isin(loyal_customers)) &(df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]
female_customers.head(10)

0     cc294636-19f0-11eb-8d74-000d3a38a36f
1     f431bd5a-ab7b-11e9-a2fc-000d3a38a36f
8     cfbda69e-5b4f-11ea-aca7-000d3a38a36f
11    c2e15af2-9eed-11e9-9897-000d3a38a36f
33    fb840306-1219-11ea-a001-000d3a38a36f
60    07ddfcfe-2590-11ea-b1c8-000d3a38a36f
74    8423eda8-ae37-11e9-a2fc-000d3a38a36f
77    45beaf52-add9-11e9-a2fc-000d3a38a36f
80    b4f0a6e6-ae60-11e9-a2fc-000d3a38a36f
83    7bb4c782-a450-11e9-a2fc-000d3a38a36f
Name: master_id, dtype: object

A 40% discount is scheduled for Men's and Children's products. The initiative aims to specifically target customers who have been valued in the past, particularly those who have not engaged in shopping for an extended period, as well as new customers expressing interest in these discounted categories.

In [28]:
target_segments_customer_ids = rfm[rfm["segment"].isin(["cant_loose","hibernating","new_customers"])]["customer_id"]
cust_ids = df[(df["master_id"].isin(target_segments_customer_ids)) & ((df["interested_in_categories_12"].str.contains("ERKEK"))|(df["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]
cust_ids.head(10)

7     3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f
10    ae608ece-c9d8-11ea-a31e-000d3a38a36f
15    13ed97a4-b167-11e9-89fa-000d3a38a36f
19    2730793e-3908-11ea-85d6-000d3a38a36f
21    7b289956-d691-11e9-93bc-000d3a38a36f
29    a46cc34a-a19f-11e9-a2fc-000d3a38a36f
35    7a52fd46-5be8-11ea-a7c9-000d3a38a36f
48    1eb32dea-a409-11e9-a2fc-000d3a38a36f
66    bb548d60-abb6-11e9-a2fc-000d3a38a36f
76    7d58deb6-62fa-11ea-a6dc-000d3a38a36f
Name: master_id, dtype: object